pl = round(((history["Close"].iloc[-1] * open_.loc[idx,"Shares"]) * (open_.loc[idx,"Exchange Rate"] * 0.99)) - open_.loc[idx,"Amount (SGD)"],2)


In [1]:
import pandas as pd
import os
import yfinance as yf

In [2]:
DIR = r"C:\Users\Ben Lau\Desktop\trading-dashboard"

In [3]:
df = pd.read_excel(os.path.join(DIR,"Transaction Data.xlsx"), sheet_name="Open Position", parse_dates=["Date"])
df = df[["Date","Symbol","Shares","Avg Price","Exchange Rate","Comm","Amount (SGD)"]].rename({"Date":"BuyDate"},axis=1)

In [4]:
# download historical data
min_date = df["BuyDate"].min()
tickers = df["Symbol"].unique()
tickers = " ".join(tickers)
tickers

'SOL1-USD ETH-USD SAND-USD CRO-USD'

In [5]:
data = yf.download(tickers, start = min_date, interval="1d", group_by="ticker").reset_index()

[*********************100%***********************]  4 of 4 completed


In [6]:
data = data.melt(id_vars="Date", var_name=["Symbol","OHLC"], value_name="PRICE")
data = data[data["OHLC"]=="Close"].drop(["OHLC"],axis=1)

In [7]:
data

,Date,Symbol,PRICE
69,2021-11-25,ETH-USD,4274.743164
70,2021-11-26,ETH-USD,4030.908936
71,2021-11-27,ETH-USD,4096.912109
72,2021-11-28,ETH-USD,4294.453613
73,2021-11-29,ETH-USD,4445.104980
...,...,...,...
501,2021-12-13,SAND-USD,4.796265
502,2021-12-14,SAND-USD,4.778493
503,2021-12-15,SAND-USD,5.146542
504,2021-12-16,SAND-USD,4.951120


In [29]:
# get daily pl
combined = pd.merge(df,data,on="Symbol")
combined = combined[combined["Date"]>=combined["BuyDate"]].copy()

In [30]:
combined["P/L (SGD)"] = ((combined["PRICE"] * combined["Shares"]) * (combined["Exchange Rate"] * 0.99)) - combined["Amount (SGD)"]
combined = combined[["Date","Symbol","Amount (SGD)","P/L (SGD)"]].copy()

In [32]:
test = combined.groupby("Date").sum().reset_index()

In [36]:
combined.to_csv("Historical PL.csv", index=False)

In [17]:
((191.511749 - 161.55873) * 6.7069 ) * 1.37 * 0.99

272.46968821671106

In [19]:
(191.511749*6.7069 * (1.37 * 0.99)) - 1484.474793

257.62494458795413